<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 3**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

**Requisitos mínimos:**
* **Actividad 1:** Path-length - (BM1 vs BM2 vs CRW) (4 pts)
* **Actividad 2:** Mean Squared Displacement - (BM vs CRW) (4 pts)
* **Actividad 3:** Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)
* **Actividad 4:** Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

**Revisión de la práctica:**
* El notebook con la práctica se entregará en un repositorio en **GitHub**. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* El estudiante deberá ser capaz de explicar su código y de corregir errores introducidos a este.

## Modules

In [60]:
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

## Classes

In [61]:
# Nota: Esta clase la importaremos junto con el segundo bloque de modulos
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## **Actividad 1: Path-length - (BM1 vs BM2 vs CRW) (4 pts)**
* Implementar función que genere **Brownian Motions (BM)** utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks (CRW)** utilizando **pandas**.
* Implementar una **función** alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [96]:
# Función para generar Brownian Motions (BM) con Pandas
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [97]:
# Función para generar Correlated Random Walks (CRW) con pandas
def generate_crw(n_steps, alpha, beta, start=(0, 0)):
    """
    Genera Correlated Random Walks (CRW) utilizando pandas.

    Parameters:
    - n_steps (int): Número de pasos en la caminata.
    - alpha (float): Parámetro de escala para la distribución normal que afecta los cambios en x.
    - beta (float): Parámetro de escala para la distribución normal que afecta los cambios en y.
    - start (tuple): Coordenadas iniciales (por defecto, (0, 0)).

    Returns:
    - pandas.DataFrame: DataFrame con las coordenadas x e y de la caminata.
    """
    np.random.seed(42)  # Fijar semilla para reproducibilidad

    # Inicializar el DataFrame
    crw_df = pd.DataFrame(index=range(n_steps), columns=['x_pos', 'y_pos'])

    # Establecer las coordenadas iniciales
    crw_df.loc[0] = start

    # Generar la CRW
    for i in range(1, n_steps):
        dx = np.random.normal(0, alpha)
        dy = beta * crw_df['x_pos'][i - 1] + np.random.normal(0, 1)

        crw_df.loc[i, 'x_pos'] = crw_df['x_pos'][i - 1] + dx
        crw_df.loc[i, 'y_pos'] = crw_df['y_pos'][i - 1] + dy

    return crw_df

In [98]:
# BM con vel 3
BM_2d_df_3 = bm_2d(1000, 3, [0,0])

# BM con vel 6
BM_2d_df_6 = bm_2d(1000, 6, [0,0])

# CRW con alpha 0.1 y beta 0.3
CRW_2d_df_9 = generate_crw(1000, 0.1, 0.3, [0,0])

# Calcular distancia entre detecciones consecutivos
dis_BM_3 = np.array([distance.euclidean(BM_2d_df_3.iloc[i-1],BM_2d_df_3.iloc[i]) for i in range(1,BM_2d_df_3.shape[0])])
# Calculo del path length
pl_BM_3 = np.cumsum(dis_BM_3)

# Calcular distancia entre detecciones consecutivas
dis_BM_6 = np.array([distance.euclidean(BM_2d_df_6.iloc[i-1],BM_2d_df_6.iloc[i]) for i in range(1,BM_2d_df_6.shape[0])])
# Calculo del path length
pl_BM_6 = np.cumsum(dis_BM_6)

# Calcular distancia entre detecciones consecutivas
dis_CRW_6 = np.array([distance.euclidean(CRW_2d_df_9.iloc[i-1],CRW_2d_df_9.iloc[i]) for i in range(1,CRW_2d_df_9.shape[0])])
# Calculo del path length
pl_CRW_6 = np.cumsum(dis_CRW_6)


In [99]:
# Inicializamos la figura
fig_path_length = go.Figure()

# Trazamos el Brownian Motion con velocidad 3 (BM_3)
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3))+1,
    y = pl_BM_3,
    line = dict(width = 2, color = 'green'),
    name = 'path_length_BM_3',
    showlegend = True
))

# Trazamos el Brownian Motion con velocidad 6 (BM_6)
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_6))+1,
    y = pl_BM_6,
    line = dict(width = 3, color = 'red'),
    name = 'path_length_BM_6',
    showlegend = True
))

# Trace CRW6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_6))+1,
    y = pl_CRW_6,
    line = dict(width = 4, color = 'yellow'),
    name = 'path_length_CRW_6',
    showlegend = True
))

fig_path_length.update_layout(title='Brownian Motion and CRW - Path Length',
                              xaxis_title='Step',
                              yaxis_title='Normalized Length')  # Actualizado el título del eje y

fig_path_length.show()

In [ ]:
"""
    Calculate the distance and cumulative length of a trajectory.

    Parameters:
    - trajectory (pandas.DataFrame): DataFrame with the coordinates of the trajectory.

    Returns:
    - tuple: A tuple containing two lists, the first is the list of distances, and the second is the list of cumulative lengths.
    """

    if trajectory.shape[0] < 2:
        raise ValueError("The trajectory must have at least two points.")

    differences = trajectory.diff().dropna()
    distances = np.linalg.norm(differences, axis=1).tolist()
    lengths = np.cumsum(distances)
    display(trajectory)
    return distances, lengths

In [82]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

def convert_to_numeric(trajectory):
    # Convertir las columnas 'x_pos' e 'y_pos' a numérico

    #df = trajectory
    #df['x_pos'] = float(df['x_pos'])
    #df['y_pos'] = float(df['y_pos'])
    #trajectory = df
    #trajectory['x_pos'] = pd.to_numeric(trajectory['x_pos'], errors='coerce')
    #trajectory['y_pos'] = pd.to_numeric(trajectory['y_pos'], errors='coerce')
    #trajectory['x_pos'] = pd.to_numeric(trajectory['x_pos'], errors='coerce')
    #trajectory['y_pos'] = pd.to_numeric(trajectory['y_pos'], errors='coerce')
    return trajectory

# Define your function to compute path length for given trajectory
def get_path_length(trajectory):
    trajectory = convert_to_numeric(trajectory)
    display(trajectory)
    distance = [np.linalg.norm(trajectory.iloc[i-1] - trajectory.iloc[i]) for i in range(1, trajectory.shape[0])]
    lengths = np.cumsum(distances)
    return distances, lengths

# Get Path length calling the function
#PL_BM_3 = get_path_length(BM_2d_df_3)

# Calcular distancia entre detecciones consecutivos
dis_BM_3 = np.array([distance.euclidean(BM_2d_df_3.iloc[i-1],BM_2d_df_3.iloc[i]) for i in range(1,BM_2d_df_3.shape[0])])
# Calculo del path length
pl_BM_3 = np.cumsum(dis_BM_3)

# Get Path length calling the function
#PL_BM_6 = get_path_length(BM_2d_df_6)

# Get Path length calling the function
#PL_CRW_6 = get_path_length(CRW_2d_df_9)

# Plotting
# Init figure
fig_path_length = go.Figure()

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3))+1,
    y = pl_BM_3,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM1
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_BM_3[1]) + 1), y=PL_BM_3[1]/max_length,
                                     mode='lines',
                                     name='BM_3', line=dict(color='red')))

# Second trace BM2
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_BM_6[1]) + 1), y=PL_BM_6[1]/max_length,
                                     mode='lines',
                                     name='BM_6', line=dict(color='green')))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_CRW_6[1]) + 1), y=PL_CRW_6[1]/max_length,
                                     mode='lines',
                                     name='CRW_9', line=dict(color='yellow')))

fig_path_length.update_layout(title='Brownian Motion and CRW - Path Length',
                              xaxis_title='Step',
                              yaxis_title='Normalized Length')  # Actualizado el título del eje y

fig_path_length.show()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

## **Actividad 2: Mean Squared Displacement - (BM vs CRW) (4 pts)**
* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en **Pandas** Data Frame.
* Visualizar con **plotly**.

## **Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Generar dos **CRWs** con dos exponentes diferentes.
* Guardar trayectorias en **pandas** Data Frame.
* Implementar una **función** que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con **plotly**.

## **Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Implementar función que genere **Lévy Walks** (LW) utilizando **pandas**.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una **función** que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener Step-length distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con **plotly**.